<h3>1. Data Loading and Preprocessing
First, let's load the images from the folders and preprocess them.</h3>

In [1]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim



In [4]:
class NumberPlateDataset(Dataset):
    def __init__(self, accepted_dir, rejected_dir, transform=None):
        self.images = []
        self.labels = []
        self.transform = transform
        
        # Load accepted images
        for img_name in os.listdir(accepted_dir):
            img_path = os.path.join(accepted_dir, img_name)
            image = cv2.imread(img_path)
            if image is not None:
                self.images.append(image)
                self.labels.append(1)  # Label for accepted images

        # Load rejected images
        for img_name in os.listdir(rejected_dir):
            img_path = os.path.join(rejected_dir, img_name)
            image = cv2.imread(img_path)
            if image is not None:
                self.images.append(image)
                self.labels.append(0)  # Label for rejected images

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = self.images[idx]
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [5]:
# Paths to your data directories
accepted_dir = 'HSRP APPROVED\HSRP APPROVED'
rejected_dir = 'hsrp reject\hsrp reject'

# Transformations for the dataset
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])


In [6]:

# Create the dataset
dataset = NumberPlateDataset(accepted_dir, rejected_dir, transform=transform)

# Split the dataset into training and testing sets
train_size = int(0.8 * len(dataset))
test_size = len(dataset) - train_size
train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


<h3>2. Model Definition
Define a CNN model for the classification task using PyTorch.</h3>

In [7]:

class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        self.fc1 = nn.Linear(128 * 16 * 16, 128)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(128, 1)
    
    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = self.pool(F.relu(self.conv3(x)))
        x = x.view(-1, 128 * 16 * 16)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = torch.sigmoid(self.fc2(x))
        return x

# Instantiate the model
model = CNNModel()


<h3>3. Training
Train the model using the training data.</h3>

In [8]:

# Loss function and optimizer
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        labels = labels.view(-1, 1).float()
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")


Epoch 1/20, Loss: 0.6961309512456259
Epoch 2/20, Loss: 0.5270646810531616
Epoch 3/20, Loss: 0.3927823454141617
Epoch 4/20, Loss: 0.40101880331834155
Epoch 5/20, Loss: 0.3882383902867635
Epoch 6/20, Loss: 0.2948625832796097
Epoch 7/20, Loss: 0.298771175245444
Epoch 8/20, Loss: 0.19134412333369255
Epoch 9/20, Loss: 0.20377719402313232
Epoch 10/20, Loss: 0.12861181050539017
Epoch 11/20, Loss: 0.1096373920639356
Epoch 12/20, Loss: 0.11415357763568561
Epoch 13/20, Loss: 0.10622553341090679
Epoch 14/20, Loss: 0.06428434420377016
Epoch 15/20, Loss: 0.05704801777998606
Epoch 16/20, Loss: 0.04517635672042767
Epoch 17/20, Loss: 0.015106623492708119
Epoch 18/20, Loss: 0.011195477660900602
Epoch 19/20, Loss: 0.012798331794328988
Epoch 20/20, Loss: 0.0046305050879406435


4. Evaluation
Evaluate the model using the test data.

In [9]:
# Evaluation
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        labels = labels.view(-1, 1).float()
        outputs = model(images)
        predicted = (outputs > 0.5).float()
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total:.2f}%")


Test Accuracy: 83.72%


<h3>Saving the Model<br>
Add the following lines at the end of your training script to save the model:</h3>

In [10]:
# Save the model
torch.save(model.state_dict(), 'number_plate_model.pth')


<h3>For Detecting the color:</h3>